# Emulation running from notebook

In [1]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

## Katrina ADCIRC set-up

In [2]:
import matplotlib.pyplot as plt
from sithom.plot import plot_defaults

plot_defaults()

/Users/simon/new-orleans/env/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [3]:
# six degrees of freedom search
from src.models.emu6d import SixDOFSearch

In [4]:
# Two degrees of freedom search [bearing and position]
from src.models.emulation import EmulationSmash